In [16]:
import xml.etree.ElementTree as ET
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

df=pd.DataFrame(columns=["id","date","title","abstract","embedding"])
with open("xmls/temp/temp.xml") as input:
    n_line=0
    temp_file = open('xmls/temp/temp.xml','w')
    while True:
        line = input.readline()
        if not line: 
            break
        if line.startswith('<?xml') and n_line!=0:
            temp_file.close()
            tree = ET.parse('xmls/temp/temp.xml')
            us_patent_grant = tree.getroot()
            title=None
            ab=None
            for publication_reference in us_patent_grant.iter("publication-reference"):
                id=publication_reference.find("document-id").find("doc-number").text
                date=publication_reference.find("document-id").find("date").text
            for us_bibliographic_data_grant in us_patent_grant.iter("us-bibliographic-data-grant"):
                title=us_bibliographic_data_grant.find("invention-title").text
            for abstract in us_patent_grant.iter("abstract"):
                ab="".join(abstract.itertext())
            if ab!=None :
                emb=model.encode(ab)
                df.loc[len(df)]=[id,date,title,ab,emb]
            temp_file = open('xmls/temp/temp.xml','w')
        temp_file.write(line)   
        n_line+=1
    df.to_csv("xmls/temp/temp.csv")

In [19]:
create_dataset("ipg230103.xml")